In [1]:
import Base: exp, sin, cos, +, -, *, /, sqrt, convert, promote_rule, zero
import Test: @test, @testset

# Define dual number

In [2]:
struct Infinitesimal{T <: Number} <: Number
    x::T
    dx::T
end

In [3]:
function convert(::Type{Infinitesimal{T}}, x::T) where {T}
   Infinitesimal(x, zero(x)) 
end
function convert(::Type{Infinitesimal{T}}, x::Infinitesimal{S}) where {S, T}
   Infinitesimal(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Infinitesimal{T}}, x::T) where {T <: Number}
    Infinitesimal(x, zero(x))
end
function convert(::Type{Infinitesimal{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Infinitesimal(x_as_T, zero(x_as_T))
end

convert (generic function with 187 methods)

In [4]:
function zero(x::Infinitesimal{T}) where T
    Infinitesimal(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [5]:
function promote_rule(::Type{Infinitesimal{T}}, ::Type{Infinitesimal{S}}) where {T,S}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{Infinitesimal{T}}, ::Type{S}) where {T, S <: Number}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Infinitesimal{S}}) where {T <: Number, S}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Infinitesimal{T}}) where {S <: AbstractIrrational, T}
    Infinitesimal{promote_type(S,T)}
end

promote_rule (generic function with 126 methods)

In [6]:
function extract_derivative(xdx::Infinitesimal)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [7]:
function D(f)
    function df(x)
        xdx = Infinitesimal(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df
end
function D(i::Integer, f)
    function df(xs...)
        xarr = [(j != i ? x : Infinitesimal(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df
end

D (generic function with 2 methods)

In [9]:
function +(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x+y.x, x.dx+y.dx)
end
function -(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x-y.x, x.dx-y.dx)
end
function -(x::Infinitesimal)
    Infinitesimal(-x.x, -x.dx)
end
function *(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end

/ (generic function with 119 methods)

In [10]:
function sqrt(x::Infinitesimal)
    Infinitesimal(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 21 methods)

In [11]:
function exp(x::Infinitesimal)
    return Infinitesimal(exp(x.x), exp(x.x)*x.dx)
end
function sin(x::Infinitesimal)
    return Infinitesimal(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Infinitesimal)
    return Infinitesimal(cos(x.x), -sin(x.x)*x.dx)
end

cos (generic function with 14 methods)

In [12]:
@test begin
    x = randn()
    isapprox(D(exp)(x), exp(x))
end

Test Passed

In [13]:
function f(x)
    return exp(-x/pi)*sin(2.0*pi*sqrt(2)*x)
end
fprime = D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [14]:
fprime(3)

0.03569811122195761

In [15]:
function f(x)
#     return exp(-x/pi)*sin(2.0*pi*sqrt(2)*x)
    return sin(x*x)
end
fprime = D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [16]:
D(2)

(::var"#df#1"{Int64}) (generic function with 1 method)